# Fran

This file created to answer the 


In [37]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import ensemble, linear_model
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

In [38]:
data_test = pd.read_csv('D:\# fran\Magang\cs-test.csv')

In [39]:
data_test.fillna(value = 0, inplace = True)

In [40]:
data_test.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.0,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,2,0.0,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,3,0.0,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,4,0.0,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,5,0.0,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


In [41]:
expected = pd.read_csv('D:\# fran\Magang\sampleEntry.csv')

In [42]:
expected.head()

,Id,Probability
0,1,0.080807
1,2,0.040719
2,3,0.011968
3,4,0.067640
4,5,0.108264


In [54]:
plt.plot(data_test)

plt.show()

In [7]:
data_train = pd.read_csv('D:\# fran\Magang\cs-training.csv')

In [10]:
data_train.fillna(value = 0, inplace = True)

In [8]:
data_train.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [9]:
reg = linear_model.LinearRegression()

In [20]:
reg.fit(data_test, expected)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [56]:
reg.predict(data_train)

array([[  1.00000000e+00,   1.59223330e-01],
       [  2.00000000e+00,   7.87212446e-02],
       [  3.00000000e+00,   1.75581675e-01],
       ..., 
       [  1.49998000e+05,   4.26550508e-02],
       [  1.49999000e+05,   9.19738438e-02],
       [  1.50000000e+05,   3.69761969e-02]])